In [1]:
import pandas as pd
import numpy as np
import matplotlib as mplot
import seaborn as sb
import time

In [2]:
movies_genres_data=pd.read_csv("Movies_genres.csv")

In [3]:
def numeric_eliminator(total_text, index, column):
        string = ""
        #print(total_text.split())
        for words in total_text.split():
            # remove the special chars in review like '"#$@!%^&*()_+-~?>< etc.
            word = ("".join(e for e in words if e.isalpha()))
            # Conver all letters to lower-case
            word = word.lower()
            string += word + " "
        movies_genres_data[column][index] = string

In [4]:
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in movies_genres_data.iterrows():
    numeric_eliminator(row['Title'], index, 'Title')
# we print the time it took to preprocess whole titles
print((time.clock() - start_time)/60, "minutes")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


6.176678787111492 minutes


In [5]:
movies_genres_data.to_csv("movies_data.csv",index=False)